# ComCamSim 2024 DIA Sprint
Authors: Michael Wood-Vasey <wmwv@pitt.edu>  
Last Verified to Run: 2024-05-17  

DIA Sprint.  This Notebook is to explore and figure out how to calculate the Metrics and visualizations we want to look at dipole orientation across an image and check against other things of interest.  Some key new quantities calculated are

"ip_diffim_DipoleFit_separation"  
"ip_diffim_DipoleFit_orientation"

These are based on the postive and negative lobs of the dipole as fit against the science (pos) and template (neg) images.

Let's see what we can learn about the orientation on the sky.

1. [x] Count dipoles and plot on image
3. [x] Look at pixel stamps of dipoles
4. [x] Spatially plot dipole orientation
5. [x] Plot w.r.t. Parallactic Angle.

Run at RSP on USDF.  Run using `d_2024_05_15` on a Large container (because we need the memory).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from lsst.daf.butler import Butler
import lsst.daf.butler as dafButler
from lsst.daf.butler import DatasetNotFoundError
from lsst.pipe.base import Instrument
from lsst.ap.association.transformDiaSourceCatalog import UnpackApdbFlags
import lsst.afw.display as afwDisplay
import lsst.display.astrowidgets
from lsst.ip.diffim.dcrModel import calculateImageParallacticAngle
import lsst.afw.image as afwImage

from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, MinMaxInterval, LogStretch
from astropy.table import Table, join

afwDisplay.setDefaultBackend('matplotlib')

Key orientation information: repo, collection, dataset types.

In [ ]:
repo = "/repo/embargo"
collection = "u/elhoward/DM-44138/LSSTComCamSim"
collections = [collection, "LSSTComCamSim/templates", "LSSTComCamSim/defaults"]
instrument = "LSSTComCamSim"

Dataset Types we'll be interested in  
`postISRCCD`  (I don't think there's a `calexp` in prompt processing)  
`goodSeeingDiff_differenceExp`  
`goodSeeingDiff_diaSrc`  (or `goodSeeingDiff_diaSrcTable` depending on the schema you want.)

In [ ]:
butler = Butler(repo, collections=collections, instrument=instrument)

In [ ]:
registry = butler.registry
for ref in registry.queryDatasets('goodSeeingDiff_differenceExp'):
    print(ref.dataId)

In [ ]:
data_id = {"detector": 0, "exposure": 7024040400021}
# An alternate data_id to look at
# data_id = {"detector": 5, "exposure": 7024040300440}
data_id["visit"] = data_id["exposure"]

In [ ]:
science = butler.get("initial_pvi", dataId=data_id)
diff = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
template = butler.get("goodSeeingDiff_templateExp", dataId=data_id)

In [ ]:
# Catalog from raw pipeline output is a `lsst.afw.table.SourceCatalog` and includes 100 blank "sky" sources
full_dia_cat = butler.get("goodSeeingDiff_diaSrc", dataId=data_id )

In [ ]:
fit_but_not_dipole = full_dia_cat[full_dia_cat["ip_diffim_DipoleFit_flag"] & ~full_dia_cat["ip_diffim_DipoleFit_flag_classification"] & ~full_dia_cat["sky_source"]].copy(deep=True)

In [ ]:
import re
ip_diffim_cols = [c for c in full_dia_cat.schema.getNames() if re.match("ip_diffim", c)]

In [ ]:
# for c in ip_diffim_cols:
#     print(c, fit_but_not_dipole[c])

In [ ]:
 # src = butler.get("src", dataId=data_id)

In [ ]:
dia_sources = full_dia_cat[full_dia_cat["sky_source"] == False]
sky_sources = full_dia_cat[full_dia_cat["sky_source"] == True]

# Deep copy to get a new contiguous catalog
dipoles = dia_sources[dia_sources["ip_diffim_DipoleFit_flag_classification"]].copy(deep=True)

In [ ]:
plt.scatter(dipoles["ip_diffim_DipoleFit_pos_instFlux"], dipoles["ip_diffim_DipoleFit_separation"])

In [ ]:
SHOW_NAMES = True
if SHOW_NAMES:
    print(full_dia_cat.schema.getNames())

In [ ]:
science.getInfo().getVisitInfo().getBoresightParAngle()

In [ ]:
print(f"{len(dia_sources)} total detections.")
print(f"{len(dipoles)} detections classified as dipoles.")

In [ ]:
psf = science.getPsf()
middle_x, middle_y = 2000, 2000
position = lsst.geom.Point2D(middle_x, middle_y)
sigma = psf.computeShape(position).getDeterminantRadius()
print(2 * sigma)

The `ip_diffim_Dipolefit_separation` is fit to the science and template and not the DIA image.  So we expect separations to be on the order of hundredths of a pixel, not the 2*sigma that the poles are separated by in the difference image.

In [ ]:
large_separation_threshold = 0.8  # pixels
unreasonable_separation_threshold = 10  # pixels
reasonable_dipoles = dipoles[dipoles["ip_diffim_DipoleFit_separation"] <= large_separation_threshold].copy(deep=True)
large_dipoles  = dipoles[(dipoles["ip_diffim_DipoleFit_separation"] > large_separation_threshold) 
                      & (dipoles["ip_diffim_DipoleFit_separation"] < unreasonable_separation_threshold)].copy(deep=True)
unbelievably_wide_dipoles = dipoles[dipoles["ip_diffim_DipoleFit_separation"] > unreasonable_separation_threshold].copy(deep=True)

In [ ]:
# The parallactic angle is in the image metadata
# For an object on the meridian this is degrees CCW of South.
# Range [-180, +180]
# https://github.com/lsst/afw/blob/b3a676b8dc86f8714a928f75276a329461bb25ed/include/lsst/afw/image/VisitInfo.h#L208
par_ang = science.getInfo().getVisitInfo().boresightParAngle.asDegrees()
# Position angle of focal plane +Y direction with respect to North
# https://github.com/lsst/afw/blob/b3a676b8dc86f8714a928f75276a329461bb25ed/include/lsst/afw/image/VisitInfo.h#L170
rot_ang = science.getInfo().getVisitInfo().boresightRotAngle.asDegrees()

# This function from the ip_diffim DCR Model code to angle the orientation of the WCS
# to get the parallactic angle with respect to afwDisplay x, y image
par_angle = calculateImageParallacticAngle(science.getInfo().getVisitInfo(), science.getWcs())

In [ ]:
bins=np.logspace(-2, 2)
plt.hist(dipoles["ip_diffim_DipoleFit_separation"], bins=bins, histtype="step", linewidth=2)
plt.xlabel("Dipole +/- separation [pixels]")
plt.xscale("log")
plt.axvline(large_separation_threshold, color="gray", linestyle="--")
plt.axvline(unreasonable_separation_threshold, color="orange", linestyle="--")

In [ ]:
bins = np.linspace(-180, +180, 19)
plt.hist(dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, label="Dipole Orientation Angle", histtype="step")
plt.hist(large_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, label="Large Dipole")
plt.hist(unbelievably_wide_dipoles["ip_diffim_DipoleFit_orientation"], bins=bins, label="Unbelievably Large Dipole")

plt.axvline(par_angle.asDegrees(), color="orange", linestyle="--", linewidth="4", label="Parallactic Angle")
plt.xlim(-180, +180)
plt.legend()

In [ ]:
# Plot direction of arrows on sky
plt.quiver(dipoles["base_SdssCentroid_x"],
           dipoles["base_SdssCentroid_y"],
           dipoles["ip_diffim_DipoleFit_separation"],
           dipoles["ip_diffim_DipoleFit_separation"],
           angles=dipoles["ip_diffim_DipoleFit_orientation"])

Anything more than 10 pixels is spurious.  Let's take a look at them.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

plt.sca(axes[0])
display_template = afwDisplay.Display(frame=fig)
display_template.scale("linear", "zscale")
display_template.image(template)

# The following code
# https://github.com/lsst/ip_diffim/blob/main/python/lsst/ip/diffim/dipoleFitTask.py#L867
# suggests that the ip_diffim_DipoleFit_orientation convention is CCW from +x.
# Plot direction of arrows on sky
plt.quiver(reasonable_dipoles["base_SdssCentroid_x"],
           reasonable_dipoles["base_SdssCentroid_y"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           angles=reasonable_dipoles["ip_diffim_DipoleFit_orientation"],
          color="orange")

plt.sca(axes[1])
display_mask = afwDisplay.Display(frame=fig)
display_mask.image(template.mask)
# Plot direction of arrows on sky
plt.quiver(reasonable_dipoles["base_SdssCentroid_x"],
           reasonable_dipoles["base_SdssCentroid_y"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           angles=reasonable_dipoles["ip_diffim_DipoleFit_orientation"],
          color="orange")
plt.quiver(large_dipoles["base_SdssCentroid_x"],
           large_dipoles["base_SdssCentroid_y"],
           large_dipoles["ip_diffim_DipoleFit_separation"],
           large_dipoles["ip_diffim_DipoleFit_separation"],
           angles=large_dipoles["ip_diffim_DipoleFit_orientation"],
          color="red")

center_x, center_y = 2000, 2000
west_to_south = -90  # quiver plots CCW from West.  Par Angle is in terms of CCW from South
plt.quiver(center_x, center_y, 10, 10, angles=par_angle.asDegrees() + west_to_south , color="green", linestyle="--", linewidth=4)

In [ ]:
# Plot direction of arrows on sky
plt.quiver(dipoles["base_SdssCentroid_x"],
           dipoles["base_SdssCentroid_y"],
           dipoles["ip_diffim_DipoleFit_separation"],
           dipoles["ip_diffim_DipoleFit_separation"],
           angles=dipoles["ip_diffim_DipoleFit_orientation"])

In [ ]:
unbelievably_wide_dipoles["base_PixelFlags_flag"]

In [ ]:
x = unbelievably_wide_dipoles["slot_Centroid_x"]
y = unbelievably_wide_dipoles["slot_Centroid_y"]
print(x, y)

In [ ]:
i = 0
# x, y flipped for imshow
y = int(unbelievably_wide_dipoles["slot_Centroid_x"][i])
x = int(unbelievably_wide_dipoles["slot_Centroid_y"][i])
size = 50  # pixels
small_diff = diff.image.array[x-size:x+size,y-size:y+size]
small_science = science.image.array[x-size:x+size,y-size:y+size]
small_template = template.image.array[x-size:x+size,y-size:y+size]

fig, axes = plt.subplots(1, 3, figsize=(16, 6))
plt.sca(axes[0])
display_diff = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(diff)

plt.sca(axes[1])
display_science = afwDisplay.Display(frame=fig)
display_diff.scale("linear", "zscale")
display_diff.mtv(science)

plt.sca(axes[2])
display_template = afwDisplay.Display(frame=fig)
display_template.scale("linear", "zscale")
display_template.mtv(template)


plt.tight_layout()
plt.show()


In [ ]:
SHOW_SCHEMA = False
if SHOW_SCHEMA:
    print(type(dia_sources))
    print(dia_sources.schema)

In [ ]:
STAMP_SIZE = 100
def postage_stamp_from_image(image, x, y, stamp_size=STAMP_SIZE):
    """
    Return postage stamp of stamp_size x stamp_size around given x, y

    x, y can be fractional, but the image will be in original pixels.
    """

    import lsst.geom as geom
    # Naive version that is wrong because it doesn't account for x, y orientation
    center = geom.Point2D(x, y)
    size = geom.Extent2I(stamp_size, stamp_size)
    foo = image.getCutout(center, size)

    return foo

In [ ]:
def show_postage_stamps(*args, arrow=None, figsize=(12, 6)):
    n = len(args)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    displays = []
    for i, image in enumerate(args):
        plt.sca(axes[i])
        display = afwDisplay.Display(frame=fig)
        display.setMaskTransparency(80)
        display.scale("linear", "zscale")
        display.mtv(image)

        displays.append(display)

        if arrow is not None:
            plt.quiver(arrow["x"], arrow["y"], arrow["sep"]*np.cos(np.deg2rad(arrow["angle"])), arrow["sep"]*np.sin(np.deg2rad(arrow["angle"])), angles="xy", scale_units="xy", scale=1, color="red")

    plt.tight_layout()
    return fig

In [ ]:
def show_dipole_stamps(dipoles):
    x = dipoles["slot_Centroid_x"]
    y = dipoles["slot_Centroid_y"]
    dipole_neg_x = dipoles["ip_diffim_DipoleFit_neg_x"]
    dipole_neg_y = dipoles["ip_diffim_DipoleFit_neg_y"]
    separation = dipoles["ip_diffim_DipoleFit_separation"]
    orientation = dipoles["ip_diffim_DipoleFit_orientation"]
    
    for xi, yi, dip_x, dip_y, sep, theta in zip(x, y, dipole_neg_x, dipole_neg_y, separation, orientation):
        # Show image name?
        d = postage_stamp_from_image(diff, xi, yi)
        s = postage_stamp_from_image(science, xi, yi)
        t = postage_stamp_from_image(template, xi, yi)
        arrow = {"x": dip_x, "y": dip_y, "sep": sep, "angle": theta}
        show_postage_stamps(d, s, t, arrow=arrow)

In [ ]:
show_dipole_stamps(unbelievably_wide_dipoles)

In [ ]:
print(data_id)

In [ ]:
display_diff.getMaskPlaneColor()

In [ ]:
show_dipole_stamps(large_dipoles)

In [ ]:
show_dipole_stamps(reasonable_dipoles)

In [ ]:
PIXEL_FLAG_LIST = [
    'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge', 'base_PixelFlags_flag_interpolated',
    'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_bad',
    'base_PixelFlags_flag_suspect', 'base_PixelFlags_flag_interpolatedCenter', 'base_PixelFlags_flag_saturatedCenter',
    'base_PixelFlags_flag_crCenter', 'base_PixelFlags_flag_suspectCenter'
]
SHAPE_FLAG_LIST = ["slot_Shape_flag"]
EDGE_FLAG_LIST = ["base_SdssCentroid_flag_edge"]
SKY_SOURCE = ["sky_source"]

FLAG_LIST = PIXEL_FLAG_LIST + SHAPE_FLAG_LIST + EDGE_FLAG_LIST + SKY_SOURCE

SNR_THRESHOLD = 5.5

def get_good_sources_idx(df, flag_list=FLAG_LIST, snr_threshold=SNR_THRESHOLD):
    bad = np.array(np.zeros_like(df), dtype=bool)
    for flag in flag_list:
        bad |= df[flag]

    snr = df["slot_ApFlux_instFlux"] / df["slot_ApFlux_instFluxErr"]
    # This is a diff so take things both above the positive SNR cut and below the negative SNR cut
    good_snr = (snr < -snr_threshold) | (snr > snr_threshold)
        
    good = ~bad
    good &= good_snr
    return good

In [ ]:
good_sources_idx = get_good_sources_idx(dia_sources)
good_sources = dia_sources[good_sources_idx]

In [ ]:
len(good_sources)

In [ ]:
x = good_sources["slot_Centroid_x"]
y = good_sources["slot_Centroid_y"]
for xi, yi in zip(x[:10], y[:10]):
    # Show image name?
    d = postage_stamp_from_image(diff, xi, yi)
    s = postage_stamp_from_image(science, xi, yi)
    t = postage_stamp_from_image(template, xi, yi)
    show_postage_stamps(d, s, t)

Look at the DIA kernel astrometric shift

In [ ]:
# From RHL

dataId = data_id

#dataId = dict(day_obs=20240404, seq_num=248, detector=8)

def get_grid_of_dia_astrometric(data_id, npt=10):
    w, h = butler.get("goodSeeingDiff_differenceExp" + ".bbox", data_id).getDimensions()
    x = np.empty((npt, npt))
    y = np.empty_like(x)
    dx = np.empty_like(x)
    dy = np.empty_like(x)

    kappa = butler.get("goodSeeingDiff_psfMatchKernel", data_id)
    
    im = afwImage.ImageD(kappa.getDimensions())
  
    hsize = kappa.getWidth()//2
    xy = np.arange(-hsize, hsize+1)

    _x0 = ix*w
    _y0 = iy*h
    for i, _x in enumerate(np.linspace(0, w-1, npt)):
        for j, _y in enumerate(np.linspace(0, h-1, npt)):
            kappa.computeImage(im, False, _x, _y)
            x[i, j] = _x
            y[i, j] = _y
            dx[i, j] = np.average(xy, weights=np.mean(im.array, axis=0))
            dy[i, j] = np.average(xy, weights=np.mean(im.array, axis=1))

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 8))

# Plot direction of arrows on sky
Q_dipole = plt.quiver(reasonable_dipoles["base_SdssCentroid_x"],
           reasonable_dipoles["base_SdssCentroid_y"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           reasonable_dipoles["ip_diffim_DipoleFit_separation"],
           angles=reasonable_dipoles["ip_diffim_DipoleFit_orientation"],
           color="orange")

Q_kernel = plt.quiver(x, y, dx, dy)
                           
                            
len = 20
plt.quiverkey(Q_dipole, 0.1, 1.04, 1e-3*len, f"{len} pixels")
plt.quiverkey(Q_kernel, 0.1, 1.04, 1e-3*len, f"{len} mas")

plt.gca().set_aspect(1);
plt.title("visit: {visit}, detector: {detector}".format(**dataId));

In [ ]:
help(plt.quiver)